# Graph Convolutional Network

In [ ]:
!pip install torch torch-geometric matplotlib scikit-learn

In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops, degree

Для решения задачи, возьмем датасет CiteSeer

In [ ]:
torch.manual_seed(42)

In [34]:
dataset = Planetoid(root='dataset/CiteSeer', name='CiteSeer', transform=NormalizeFeatures())

In [35]:
data = dataset[0]

In [46]:
dataset.num_features, dataset.num_classes

(3703, 6)

Построим GCN модель с помощью слоев GCNConv из библиотеки PyG

In [36]:
class GCN(torch.nn.Module):
    def __init__(self, hidden_dim, input_dim, output_dim, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.conv4 = GCNConv(hidden_dim, output_dim)

        self.dropout = dropout

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = F.dropout(x, p=self.dropout)

        x = self.conv2(x, edge_index).relu()
        x = F.dropout(x, p=self.dropout)

        x = self.conv3(x, edge_index).relu()
        x = F.dropout(x, p=self.dropout)

        x = self.conv4(x, edge_index)
        return x

In [37]:
def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

@torch.no_grad()
def evaluate(model, data, mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    correct = (pred[mask] == data.y[mask]).float().sum()
    accuracy = correct / mask.sum()
    return accuracy.item()

In [38]:
model = GCN(input_dim=data.num_node_features,
            hidden_dim=16,
            output_dim=dataset.num_classes)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

In [39]:
for epoch in range(300):
    train_loss = train(model, data, optimizer)
    if epoch % 20 == 0:
        val_acc = evaluate(model, data, data.val_mask)
        print(f'Epoch: {epoch:03d}, Loss: {train_loss:.4f}, Val Acc: {val_acc:.4f}')

test_acc = evaluate(model, data, data.test_mask)
print(f'Final Test Accuracy: {test_acc:.4f}')

Epoch: 000, Loss: 1.7916, Val Acc: 0.2480
Epoch: 020, Loss: 1.5815, Val Acc: 0.3000
Epoch: 040, Loss: 0.8672, Val Acc: 0.4060
Epoch: 060, Loss: 0.6519, Val Acc: 0.4460
Epoch: 080, Loss: 0.4870, Val Acc: 0.4660
Epoch: 100, Loss: 0.3787, Val Acc: 0.5340
Epoch: 120, Loss: 0.4227, Val Acc: 0.5040
Epoch: 140, Loss: 0.3468, Val Acc: 0.5180
Epoch: 160, Loss: 0.4290, Val Acc: 0.5440
Epoch: 180, Loss: 0.2484, Val Acc: 0.5660
Epoch: 200, Loss: 0.2077, Val Acc: 0.5660
Epoch: 220, Loss: 0.2233, Val Acc: 0.5780
Epoch: 240, Loss: 0.3789, Val Acc: 0.5780
Epoch: 260, Loss: 0.2427, Val Acc: 0.5560
Epoch: 280, Loss: 0.1537, Val Acc: 0.5500
Final Test Accuracy: 0.5410


При изначальных гиперпараметрах `lr=0.01, hidden_dim=16, dropout=0.5 ` самый большой accuracy вышел 0.578

Теперь подберем гиперпараметры и попытаемся получить accuracy повыше

In [40]:
hidden_dim = [16, 32, 64]
lrs = [0.01, 0.001, 0.0001]
dropouts = [0.3, 0.5, 0.7]

In [41]:
best_acc = 0
best_hidden = 0
best_lr = 0
best_dropout = 0

for hidden in hidden_dim:
    for lr in lrs:
      for dr in dropouts:
          model = GCN(input_dim=data.num_node_features,
                      hidden_dim=hidden,
                      output_dim=dataset.num_classes,
                      dropout=dr
                      )
          optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)

          for epoch in range(200):
              train(model, data, optimizer)

          acc = evaluate(model, data, data.val_mask)

          print(f"{acc}: hidden {hidden}, lr {lr}, dropout {dr}")

          if acc > best_acc:
              best_acc = acc
              best_hidden = hidden
              best_lr = lr
              best_dropout = dr

print(f"Best accuracy {best_acc:.4f} with hidden_dim: {best_hidden}, lr: {best_lr}, dropout: {best_dropout}")

0.5419999957084656: hidden 16, lr 0.01, dropout 0.3
0.5559999942779541: hidden 16, lr 0.01, dropout 0.5
0.39399999380111694: hidden 16, lr 0.01, dropout 0.7
0.5360000133514404: hidden 16, lr 0.001, dropout 0.3
0.35600000619888306: hidden 16, lr 0.001, dropout 0.5
0.32600000500679016: hidden 16, lr 0.001, dropout 0.7
0.3799999952316284: hidden 16, lr 0.0001, dropout 0.3
0.29600000381469727: hidden 16, lr 0.0001, dropout 0.5
0.24400000274181366: hidden 16, lr 0.0001, dropout 0.7
0.5899999737739563: hidden 32, lr 0.01, dropout 0.3
0.5460000038146973: hidden 32, lr 0.01, dropout 0.5
0.550000011920929: hidden 32, lr 0.01, dropout 0.7
0.5680000185966492: hidden 32, lr 0.001, dropout 0.3
0.4480000138282776: hidden 32, lr 0.001, dropout 0.5
0.4580000042915344: hidden 32, lr 0.001, dropout 0.7
0.4560000002384186: hidden 32, lr 0.0001, dropout 0.3
0.35199999809265137: hidden 32, lr 0.0001, dropout 0.5
0.28600001335144043: hidden 32, lr 0.0001, dropout 0.7
0.5419999957084656: hidden 64, lr 0.01, 

В итоге самый лучший результат (Accuracy 0.6080) вышел с набором гиперпараметров `hidden_dim: 64, lr: 0.001, dropout: 0.5`

Сделаем свою реализацию GCNConv слоев (используя матричные операции), чтобы построить новую GCN-модель

In [43]:
class NewGCNConv(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.lin = torch.nn.Linear(input_dim, output_dim, bias=False)
        self.bias = nn.Parameter(torch.zeros(output_dim))
        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        nn.init.zeros_(self.bias)

    def forward(self, x, edge_index):
        x = self.lin(x)

        num_nodes=x.size(0)
        edge_index, _ = add_self_loops(edge_index, num_nodes)
        row, col = edge_index
        deg = degree(row, num_nodes, dtype=x.dtype)
        deg_inv_sqrt = deg.pow(-0.5)
        deg_inv_sqrt[deg_inv_sqrt == float('inf')] = 0
        norm = deg_inv_sqrt[row] * deg_inv_sqrt[col]

        adj = torch.sparse_coo_tensor(
            edge_index,
            norm,
            (num_nodes, num_nodes)
        )
        x = torch.sparse.mm(adj, x)

        return x + self.bias

In [44]:
class NewGCN(torch.nn.Module):
    def __init__(self, hidden_dim, input_dim, output_dim, dropout):
        super().__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.conv4 = GCNConv(hidden_dim, output_dim)
        self.dropout = dropout

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        x = F.dropout(x, p=self.dropout)

        x = self.conv2(x, edge_index).relu()
        x = F.dropout(x, p=self.dropout)

        x = self.conv3(x, edge_index).relu()
        x = F.dropout(x, p=self.dropout)

        x = self.conv4(x, edge_index)
        return x

Подберем гиперпараметры для новой реализации

In [45]:
best_acc = 0
best_hidden = 0
best_lr = 0
best_dropout = 0

for hidden in hidden_dim:
    for lr in lrs:
      for dr in dropouts:
          model = NewGCN(input_dim=data.num_node_features,
                         hidden_dim=hidden,
                         output_dim=dataset.num_classes,
                         dropout=dr
                      )
          optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)

          for epoch in range(200):
              train(model, data, optimizer)

          acc = evaluate(model, data, data.val_mask)

          print(f"{acc}: hidden {hidden}, lr {lr}, dropout {dr}")

          if acc > best_acc:
              best_acc = acc
              best_hidden = hidden
              best_lr = lr
              best_dropout = dr

print(f"Best accuracy {best_acc:.4f} with hidden_dim: {best_hidden}, lr: {best_lr}, dropout: {best_dropout}")

0.527999997138977: hidden 16, lr 0.01, dropout 0.3
0.5099999904632568: hidden 16, lr 0.01, dropout 0.5
0.36000001430511475: hidden 16, lr 0.01, dropout 0.7
0.4699999988079071: hidden 16, lr 0.001, dropout 0.3
0.35600000619888306: hidden 16, lr 0.001, dropout 0.5
0.39399999380111694: hidden 16, lr 0.001, dropout 0.7
0.30000001192092896: hidden 16, lr 0.0001, dropout 0.3
0.31200000643730164: hidden 16, lr 0.0001, dropout 0.5
0.20600000023841858: hidden 16, lr 0.0001, dropout 0.7
0.6060000061988831: hidden 32, lr 0.01, dropout 0.3
0.6019999980926514: hidden 32, lr 0.01, dropout 0.5
0.515999972820282: hidden 32, lr 0.01, dropout 0.7
0.593999981880188: hidden 32, lr 0.001, dropout 0.3
0.5600000023841858: hidden 32, lr 0.001, dropout 0.5
0.5419999957084656: hidden 32, lr 0.001, dropout 0.7
0.38999998569488525: hidden 32, lr 0.0001, dropout 0.3
0.35199999809265137: hidden 32, lr 0.0001, dropout 0.5
0.2680000066757202: hidden 32, lr 0.0001, dropout 0.7
0.6359999775886536: hidden 64, lr 0.01, d

В NewGCN получили даже лучше Accuracy 0.6360 с гиперпараметрами `hidden_dim: 64, lr: 0.01, dropout: 0.3`, чем у обычного GCN

В целом у GCN (слои из PyG) и MyGCN (слои сделали самостоятельно) получились довольно схожие результаты, что может говорить о правильности нашей реализации GCN-слоя